In [1]:
from dotenv import load_dotenv
import os
from sqlalchemy.engine import URL, Engine
from sqlalchemy import create_engine, Table, MetaData, Column, inspect
from sqlalchemy.dialects import postgresql
from jinja2 import Environment, FileSystemLoader, Template
import pandas as pd

from assets.trade_etl import load, extract_symbols, extract_trade, transform
from connectors.trade_api import MarketstackApiClient
from connectors.postgresql import PostgreSqlClient
from assets.helpers import get_schema_metadata


In [3]:
# Load enviroment variables
load_dotenv()

# Source Database variables
SOURCE_DATABASE_NAME = os.environ.get("SOURCE_DATABASE_NAME")
SOURCE_SERVER_NAME = os.environ.get("SOURCE_SERVER_NAME")
SOURCE_DB_USERNAME = os.environ.get("SOURCE_DB_USERNAME")
SOURCE_DB_PASSWORD = os.environ.get("SOURCE_DB_PASSWORD")
SOURCE_PORT = os.environ.get("SOURCE_PORT")

# Target Database Variables
TARGET_DATABASE_NAME = os.environ.get("TARGET_DATABASE_NAME")
TARGET_SERVER_NAME = os.environ.get("TARGET_SERVER_NAME")
TARGET_DB_USERNAME = os.environ.get("TARGET_DB_USERNAME")
TARGET_DB_PASSWORD = os.environ.get("TARGET_DB_PASSWORD")
TARGET_PORT = os.environ.get("TARGET_PORT")

# API key for Markets API
API_KEY = os.environ.get("MARKET_APIKEY")

# Target Table name
table_name = os.environ.get("TARGET_TABLE_NAME")

# Create the clients for the connections
Postgre_Client = PostgreSqlClient(SOURCE_SERVER_NAME, SOURCE_DATABASE_NAME, SOURCE_DB_USERNAME, SOURCE_DB_PASSWORD, SOURCE_PORT)
API_Client = MarketstackApiClient(API_KEY)

# Extract the symbols in our database table
symbols = extract_symbols(Postgre_Client)

# Extract the stock information for the symbols in our database
stock_information = extract_trade(API_Client, symbols)

# Transform the stock information
stock_information = transform(stock_information)
stock_information_df = stock_information.copy(deep=True)

# Debug Only
#stock_information.to_csv("stock_information2.csv", index=False)
#stock_information = pd.read_csv("stock_information2.csv")

# Transform the Pandas Dataframe into a list of dictionaries
stock_information["volume"] = stock_information["volume"].astype('Int64')
stock_information["date"] = stock_information["date"].astype('str')
stock_information["etl_load_timestamp"] = stock_information["etl_load_timestamp"].astype('str')
print(stock_information.info())
stock_information = stock_information.to_dict('records')
#print(stock_information)
print("#"*1000)

# Load the stock data into a table in our database
source_metadata = get_schema_metadata(engine=Postgre_Client.engine)

load(
            data=stock_information,
            table_name=table_name,
            engine=Postgre_Client.engine,
            source_metadata=source_metadata
        )
    

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 99 to 100
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                400 non-null    float64
 1   high                400 non-null    float64
 2   low                 400 non-null    float64
 3   close               400 non-null    float64
 4   volume              400 non-null    Int64  
 5   name                400 non-null    object 
 6   exchange_code       316 non-null    object 
 7   asset_type          400 non-null    object 
 8   price_currency      400 non-null    object 
 9   symbol              400 non-null    object 
 10  date                400 non-null    object 
 11  etl_load_timestamp  400 non-null    object 
 12  data_source         400 non-null    object 
dtypes: Int64(1), float64(4), object(8)
memory usage: 44.1+ KB
None
###########################################################################################

g:\projects\de_boot\project1\04-project-1\marketstack_etl\assets\trade_etl.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trade["date"] = pd.to_datetime(df_trade["date"])


ProgrammingError: (pg8000.dbapi.ProgrammingError) {'S': 'ERROR', 'V': 'ERROR', 'C': '23502', 'M': 'el valor nulo en la columna «exchange_code» de la relación «stock_information» viola la restricción de no nulo', 'D': 'La fila que falla contiene (193.65, 194.05, 190.84, 192.58, 38084900, Alphabet Inc. Cl A, null, Stock, CLP, GOOGL, 2025-07-27 21:00:00-03, 2025-10-06 10:53:41.157671-03, marketstack_api).', 's': 'public', 't': 'stock_information', 'c': 'exchange_code', 'F': 'execMain.c', 'L': '1987', 'R': 'ExecConstraints'}
[SQL: INSERT INTO stock_information (open, high, low, close, volume, name, exchange_code, asset_type, price_currency, symbol, date, etl_load_timestamp, data_source) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s), (%s, %s, %s, %s, %s, %s, %s, %s

In [ ]:
stock_information_df2 = stock_information_df.reset_index(drop=True)


# Transform the Pandas Dataframe into a list of dictionaries
stock_information_df2["volume"] = stock_information_df2["volume"].astype('Int64')
stock_information_df2["date"] = stock_information_df2["date"].astype('str')
stock_information_df2["etl_load_timestamp"] = stock_information_df2["etl_load_timestamp"].astype('str')

stock_information = stock_information_df2.to_dict('records')
#print(stock_information)


# Load the stock data into a table in our database
source_metadata = get_schema_metadata(engine=Postgre_Client.engine)

load(
            data=stock_information,
            table_name=table_name,
            engine=Postgre_Client.engine,
            source_metadata=source_metadata
        )

########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [15]:
stock_information_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                400 non-null    float64
 1   high                400 non-null    float64
 2   low                 400 non-null    float64
 3   close               400 non-null    float64
 4   volume              400 non-null    Int64  
 5   name                400 non-null    object 
 6   exchange_code       316 non-null    object 
 7   asset_type          400 non-null    object 
 8   price_currency      400 non-null    object 
 9   symbol              400 non-null    object 
 10  date                400 non-null    object 
 11  etl_load_timestamp  400 non-null    object 
 12  data_source         400 non-null    object 
dtypes: Int64(1), float64(4), object(8)
memory usage: 41.1+ KB


In [23]:
stock_information_df2.to_dict('records')

[{'open': 210.43,
  'high': 213.4,
  'low': 209.0,
  'close': 212.93,
  'volume': 51759900,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-13 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:53:41.157671',
  'data_source': 'marketstack_api'},
 {'open': 212.43,
  'high': 213.94,
  'low': 210.58,
  'close': 212.33,
  'volume': 49253000,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-14 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:53:41.157671',
  'data_source': 'marketstack_api'},
 {'open': 210.95,
  'high': 212.96,
  'low': 209.54,
  'close': 211.45,
  'volume': 44979900,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-15 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:53:41.1576

In [ ]:
df = pd.read_csv("stock_information2.csv")

In [22]:
df.to_dict('records')

[{'open': 210.43,
  'high': 213.4,
  'low': 209.0,
  'close': 212.93,
  'volume': 51759900.0,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-13 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:28:32.648424',
  'data_source': 'marketstack_api'},
 {'open': 212.43,
  'high': 213.94,
  'low': 210.58,
  'close': 212.33,
  'volume': 49253000.0,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-14 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:28:32.648424',
  'data_source': 'marketstack_api'},
 {'open': 210.95,
  'high': 212.96,
  'low': 209.54,
  'close': 211.45,
  'volume': 44979900.0,
  'name': 'Apple Inc',
  'exchange_code': 'NASDAQ',
  'asset_type': 'Stock',
  'price_currency': 'USD',
  'symbol': 'AAPL',
  'date': '2025-05-15 00:00:00+00:00',
  'etl_load_timestamp': '2025-10-06 10:28:3